# Parámetros en Pyomo

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_folder = r'Styles_for_JN'
css_style = 'style_1.css'
css_file = css_folder + '\\' + css_style
HTML(open(css_file, "r").read())

## Importes

Se importa Pyomo:

In [2]:
from pyomo.environ import *

## Declaraciones

Para declarar un parámetro en un modelo, primero debe crease el modelo:

In [3]:
model = ConcreteModel()

La declaración más simple es:

In [4]:
model.Z = Param()

In [5]:
model.Z.pprint()

Z : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
ERROR: evaluating object as numeric value: Z
        (object: <class 'pyomo.core.base.param.SimpleParam'>)
    Error evaluating Param value (Z):
    	The Param value is undefined and no default value is specified
    Key : Value
    None :  None


Se pueden proporcionar parámetros indexados. Así, creamos el índice:

In [6]:
model.Index = Set(initialize = ['Primer', 'Segun', 'Terce'], ordered = True)

Indicamos que el parámetro depende de un índice:

In [7]:
model.ZIndex = Param(model.Index)
model.ZIndex.pprint()

ZIndex : Size=0, Index=Index, Domain=Any, Default=None, Mutable=False
    Key : Value


Aqui no tiene valor. Podemos indicarle un valor al igual que un set. Inicializarlo.

In [8]:
model.ZIni = Param(model.Index, initialize = {'Primer':32, 'Segun': 23,'Terce': 21})
model.ZIni.pprint()

ZIni : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    Primer :    32
     Segun :    23
     Terce :    21


La inicialización también puede hacerse mediante una palabra clave:

In [9]:
def Initialization(model, i):
    return i + ' Parametro'
model.ZRule = Param(model.Index, initialize = Initialization)
model.ZRule.pprint()

ZRule : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    Primer : Primer Parametro
     Segun : Segun Parametro
     Terce : Terce Parametro


Se puede referir a previos parámetros en el modelo esa regla, incluso, pero el set debe ser ordenado:

In [10]:
def Previo(model, i):
    if i == 'Primer':
        return 'Primer' + i
    return i + model.ZPrevio['Primer']
model.ZPrevio = Param(model.Index, initialize = Previo)
model.ZPrevio.pprint()

ZPrevio : Size=3, Index=Index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    Primer :      PrimerPrimer
     Segun : SegunPrimerPrimer
     Terce : TercePrimerPrimer


Los parámetros pueden contener cualquier tipo de datos. El dominio por defecto es `Any`. En ocasiones, es mejor marcar un dominio para validar que lo que estamos introduciendo es correcto:

In [11]:
model.ZReal = Param(within = Reals)

## Mutabilidad

Los parámetros representan valores constantes. Sin embargo, es posible utilizar parámetros mutables en Pyomo. Por ejemplo, generaremos la siguiente expresión:

$$ x_{1} + 17x_{2} - 4x_{3}$$

In [12]:
model = ConcreteModel()
model.A = Set(initialize = [1,2,3])
model.P = Param(model.A, initialize = {1:1, 2:17, 3:-4})
model.x = Var(model.A, within = NonNegativeReals)
def Obj_Rule(model):
    return sum(model.P[i]*model.x[i] for i in model.A)
model.Obj = Objective(rule = Obj_Rule)

In [13]:
model.Obj.pprint()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : x[1] + 17*x[2] - 4*x[3]


Observamos que ha substituido el valor de los parámetros. Mutabilidad está en cero. Ahora probamos a hacerlo activando la mutabilidad.

In [14]:
model = ConcreteModel()
model.A = Set(initialize = [1,2,3])
model.P = Param(model.A, initialize = {1:1, 2:17, 3:-4}, mutable = True)
model.x = Var(model.A, within = NonNegativeReals)
def Obj_Rule(model):
    return sum(model.P[i]*model.x[i] for i in model.A)
model.Obj = Objective(rule = Obj_Rule)

In [15]:
model.Obj.pprint()

Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : P[1] * x[1] + P[2] * x[2] + P[3] * x[3]


Ahora no ha substituido el valor de los parámetros . Y estos pueden cambiarse con expresiones como:

In [16]:
model.P[2] = 4.2